# Debug Entanglement Import Issues

This notebook investigates and resolves the `ModuleNotFoundError: No module named 'show_env'` error that's preventing the persona entanglement tests from running.

## Problem Summary

When running `pytest`, the collection phase fails with:
```
src\presentation\tests\test_context_persona_entanglement.py:26: in <module>
    from admin.show_context import main as show_context_main
admin\show_context.py:17: in <module>
    from show_env import show_env_file
E   ModuleNotFoundError: No module named 'show_env'
```

The issue chain:
1. **Test modules** import `admin.show_context`
2. **show_context.py** tries to import `show_env` 
3. **show_env.py** exists in the same admin directory but isn't found

Let's debug and fix this entanglement issue step by step.

## 1. Review Project Directory Structure

First, let's examine the project structure to understand where files are located and how imports should work.

In [ ]:
import os
import sys
from pathlib import Path

# Set up project root path
ROOT = Path(__file__).resolve().parent.parent
print(f"Project root: {ROOT}")
print(f"Current working directory: {os.getcwd()}")
print(f"Python path: {sys.path[:3]}...")  # First 3 entries

# Check admin directory contents
admin_dir = ROOT / "admin"
print(f"\nAdmin directory: {admin_dir}")
print(f"Admin directory exists: {admin_dir.exists()}")

if admin_dir.exists():
    print("\nAdmin directory contents:")
    for item in sorted(admin_dir.iterdir()):
        if item.is_file() and item.suffix == '.py':
            print(f"  📄 {item.name}")

# Check specifically for the problematic files
show_context_py = admin_dir / "show_context.py"
show_env_py = admin_dir / "show_env.py"

print(f"\nFile existence check:")
print(f"  show_context.py: {show_context_py.exists()}")
print(f"  show_env.py: {show_env_py.exists()}")
print(f"  show_config.py: {(admin_dir / 'show_config.py').exists()}")
print(f"  check_env_vars.py: {(admin_dir / 'check_env_vars.py').exists()}")
print(f"  show_dependencies.py: {(admin_dir / 'show_dependencies.py').exists()}")

## 2. Reproduce ImportError in Test

Now let's simulate the exact import chain that's failing in the test suite.

# Debug Entanglement Import Errors

The entanglement tests are failing because of import errors in `admin.show_context.py`. This notebook investigates and fixes the issue.

## The Problem
```
ModuleNotFoundError: No module named 'show_env'
```

The error occurs when pytest tries to collect tests that import from `admin.show_context`, which in turn tries to import sibling modules without proper path setup.

## Investigation Plan
1. Reproduce the ImportError in a minimal example
2. Inspect sys.path and module resolution
3. Test relative vs absolute imports
4. Patch imports for test discovery
5. Validate test discovery fixes
6. Automate sys.path correction for future test runs